# Circulation Analysis

This notebook determines the maximum overtruning and barotropic circulartion for each model at the end of the Ocean1 and Ocean2 runs.

In [1]:
#Load required packages
import xarray as xr
import numpy as np
import netCDF4

Models: 
```
COCO   MITgcm-BAS          MITgcm-JPL  MOM6_SIGMA_ZSTAR  NEMO-CNRS      POP2x
FVCOM  MITgcm_BAS_Coupled  MOM6        MPAS-Ocean        NEMO-UKESM1is  ROMS-UTAS
```

In [4]:
## folder where the isomip-plus github repo is located
baserepo = '/home/xylar/Documents/Manuscripts/2024/isomip_plus/circulation-analysis/'
## folder where you downloaded the zipped Google Drive data
basedrive = '//home/xylar/data/isomip_plus/ISOMIP+Data2/'

# load the data from the text files, which point to the right location of the file in the drive structure
Ocean0_COM = np.loadtxt(baserepo+'notebooks/file-pointers/Ocean0-COM.txt', dtype = 'str', delimiter = ',',usecols = 0)
Ocean1_COM = np.loadtxt(baserepo+'notebooks/file-pointers/Ocean1-COM.txt', dtype = 'str', delimiter = ',',usecols = 0)
Ocean2_COM = np.loadtxt(baserepo+'notebooks/file-pointers/Ocean2-COM.txt', dtype = 'str', delimiter = ',',usecols = 0)
Ocean0_TYP = np.loadtxt(baserepo+'notebooks/file-pointers/Ocean0-TYP.txt', dtype = 'str', delimiter = ',',usecols = 0)
Ocean1_TYP = np.loadtxt(baserepo+'notebooks/file-pointers/Ocean1-TYP.txt', dtype = 'str', delimiter = ',',usecols = 0)
Ocean2_TYP = np.loadtxt(baserepo+'notebooks/file-pointers/Ocean2-TYP.txt', dtype = 'str', delimiter = ',',usecols = 0)


# Max Overturning Streamfunction

In [24]:
expt = Ocean1_COM
labels = []
for fileIndex in range(len(expt)):
    fileName = expt[fileIndex]
    baseName = os.path.basename(basedrive +fileName)
    modelName = ' '.join(baseName.split('_')[2:]).split('.')[0]
    modelName = modelName.strip('V3')
    labels.append(modelName)
length = len(expt)
time = 240
netcdf_fill = 9.96921e+36
for i in np.arange(length):
    data = xr.open_dataset(basedrive+expt[i], decode_times=False)
    # final year
    osf_last_year = data.overturningStreamfunction.isel(nTime = np.arange(time-13,time))
    # mask and convert to Sv
    osf_last_year = osf_last_year.where(osf_last_year < netcdf_fill)/1e6
    # mean over the final year
    osf = osf_last_year.mean('nTime')
    label = f'{labels[i]}:'
    print(f'{label: <20} {osf.max().values:.3g} Sv')


COCO:                0.385 Sv
FVCOM:               0.135 Sv
MITgcm BAS:          0.243 Sv
MITgcm BAS Coupled:  0.278 Sv
MITgcm JPL:          0.234 Sv
MOM6 SIGMA ZSTAR:    0.217 Sv
MOM6:                0.19 Sv
MPAS-Ocean:          0.208 Sv
NEMO-CNRS:           0.247 Sv
NEMO-UKESM1is:       0.207 Sv
POP2x:               0.273 Sv
ROMSUTAS:            0.0935 Sv


In [46]:
expt = Ocean2_COM
labels = []
for fileIndex in range(len(expt)):
    fileName = expt[fileIndex]
    baseName = os.path.basename(basedrive +fileName)
    modelName = ' '.join(baseName.split('_')[2:]).split('.')[0]
    modelName = modelName.strip('V3')
    labels.append(modelName)
length = len(expt)
time = 240
netcdf_fill = 9.96921e+36
mean_max = 0
mean_min = 0
mean_count = 0
coco_max = None
fvcom_max = None
for i in np.arange(length):
    data = xr.open_dataset(basedrive+expt[i], decode_times=False)
    # final year
    osf_last_year = data.overturningStreamfunction.isel(nTime = np.arange(time-13,time))
    # mask and convert to Sv
    osf_last_year = osf_last_year.where(osf_last_year < netcdf_fill)/1e6
    # mean over the final year
    osf = osf_last_year.mean('nTime')
    label = f'{labels[i]}:'
    osf_max = osf.max().values
    osf_min = osf.min().values
    if labels[i] != 'COCO' and labels[i] != 'FVCOM':
        mean_max += osf_max
        mean_min += osf_min
        mean_count += 1
    if labels[i] == 'COCO':
        coco_max = osf_max
    if labels[i] == 'FVCOM':
        fvcom_max = osf_max
    print(f'{label: <20} {osf_max:.3g} Sv')
    print(f'{"": <19} {osf_min:.3g} Sv')

mean_max = mean_max/mean_count
mean_min = mean_min/mean_count
print('Mean max     Mean min')
print(f'{1e3*mean_max:.2g} {1e3*mean_min:.2g} mSv')

print('COCO max factor   FVCOM max factor')
print(f'{coco_max/mean_max:.2g} {fvcom_max/mean_max:.2g}')



COCO:                0.117 Sv
                    -0.0517 Sv
FVCOM:               0.0266 Sv
                    -0.0186 Sv
MITgcm BAS:          0.0087 Sv
                    -0.00693 Sv
MITgcm BAS Coupled:  0.00841 Sv
                    -0.00795 Sv
MITgcm JPL:          0.00667 Sv
                    -0.00819 Sv
MOM6 SIGMA ZSTAR:    0.00841 Sv
                    -0.0148 Sv
MOM6:                0.0082 Sv
                    -0.00587 Sv
MPAS-Ocean:          0.00898 Sv
                    -0.00614 Sv
NEMO-CNRS:           0.00768 Sv
                    -0.00523 Sv
NEMO-UKESM1is:       0.00715 Sv
                    -0.00861 Sv
POP2x:               0.0076 Sv
                    -0.00626 Sv
ROMSUTAS:            0.0104 Sv
                    -0.00606 Sv
Mean max     Mean min
8.2 -7.6 mSv
COCO max factor   FVCOM max factor
14 3.2


# Max BarogropicStreamfunction

In [38]:
expt = Ocean1_COM
labels = []
for fileIndex in range(len(expt)):
    fileName = expt[fileIndex]
    baseName = os.path.basename(basedrive +fileName)
    modelName = ' '.join(baseName.split('_')[2:]).split('.')[0]
    modelName = modelName.strip('V3')
    labels.append(modelName)
length = len(expt)
time = 240
netcdf_fill = 9.96921e+36
regions = {'Calving Face': [630., 650., 0., 80.],
           'Western Boundary': [500., 620., 50., 80.]}
for region_name, bounds in regions.items():
    print(f'\n{region_name}:')
    max_val = None
    min_val = None
    for i in np.arange(length):
        data = xr.open_dataset(basedrive+expt[i], decode_times=False)
        # final year
        bsf_last_year = data.barotropicStreamfunction.isel(nTime = np.arange(time-13,time))
        # mask and convert to Sv
        bsf_last_year = bsf_last_year.where(bsf_last_year < netcdf_fill)/1e6
        # mean over the final year
        bsf = bsf_last_year.mean('nTime')
        x = bsf.nx * 2. + 321.
        y = bsf.ny * 2. + 1
        bsf = bsf.where(x >= bounds[0])
        bsf = bsf.where(x <= bounds[1])
        bsf = bsf.where(y >= bounds[2])
        bsf = bsf.where(y <= bounds[3])
        label = f'{labels[i]}:'
        max_bsf = bsf.max().values
        if labels[i] != 'COCO':
            if max_val is None or max_val < max_bsf:
                max_val = max_bsf
            if min_val is None or min_val > max_bsf:
                min_val = max_bsf
        print(f'   {label: <20} {max_bsf:.3g} Sv')

    print(f'   {0.5 * (min_val + max_val):.2g} $\pm$ {0.5 * (max_val - min_val):.2g} Sv')



Calving Face:
   COCO:                0.948 Sv
   FVCOM:               0.391 Sv
   MITgcm BAS:          0.26 Sv
   MITgcm BAS Coupled:  0.682 Sv
   MITgcm JPL:          0.279 Sv
   MOM6 SIGMA ZSTAR:    0.417 Sv
   MOM6:                0.144 Sv
   MPAS-Ocean:          0.499 Sv
   NEMO-CNRS:           0.607 Sv
   NEMO-UKESM1is:       0.23 Sv
   POP2x:               0.572 Sv
   ROMSUTAS:            0.245 Sv
   0.41 $\pm$ 0.27 Sv

Western Boundary:
   COCO:                0.178 Sv
   FVCOM:               0.286 Sv
   MITgcm BAS:          0.269 Sv
   MITgcm BAS Coupled:  0.429 Sv
   MITgcm JPL:          0.246 Sv
   MOM6 SIGMA ZSTAR:    0.251 Sv
   MOM6:                0.176 Sv
   MPAS-Ocean:          0.223 Sv
   NEMO-CNRS:           0.256 Sv
   NEMO-UKESM1is:       0.237 Sv
   POP2x:               0.232 Sv
   ROMSUTAS:            0.0926 Sv
   0.26 $\pm$ 0.17 Sv


In [52]:
expt = Ocean2_COM
labels = []
for fileIndex in range(len(expt)):
    fileName = expt[fileIndex]
    baseName = os.path.basename(basedrive +fileName)
    modelName = ' '.join(baseName.split('_')[2:]).split('.')[0]
    modelName = modelName.strip('V3')
    labels.append(modelName)
length = len(expt)
time = 240
netcdf_fill = 9.96921e+36
regions = {'Grounding Line': [400., 460., 0., 80.]}
for region_name, bounds in regions.items():
    print(f'\n{region_name}:')
    max_val = None
    min_val = None
    for i in np.arange(length):
        data = xr.open_dataset(basedrive+expt[i], decode_times=False)
        # final year
        bsf_last_year = data.barotropicStreamfunction.isel(nTime = np.arange(time-13,time))
        # mask and convert to Sv
        bsf_last_year = bsf_last_year.where(bsf_last_year < netcdf_fill)/1e6
        # mean over the final year
        bsf = bsf_last_year.mean('nTime')
        x = bsf.nx * 2. + 321.
        y = bsf.ny * 2. + 1
        bsf = bsf.where(x >= bounds[0])
        bsf = bsf.where(x <= bounds[1])
        bsf = bsf.where(y >= bounds[2])
        bsf = bsf.where(y <= bounds[3])
        label = f'{labels[i]}:'
        max_bsf = bsf.max().values
        min_bsf = bsf.min().values
        if labels[i] != 'COCO':
            if max_val is None or max_val < max_bsf:
                max_val = max_bsf
            if min_val is None or min_val > max_bsf:
                min_val = max_bsf
        print(f'   {label: <20} {max_bsf:.3g} Sv')
        print(f'   {"": <19} {min_bsf:.3g} Sv')

    print(f'   {0.5 * (min_val + max_val):.2g} $\pm$ {0.5 * (max_val - min_val):.2g} Sv')

regions = {'Outer Cavity': [460., 660., 0., 80.]}
for region_name, bounds in regions.items():
    print(f'\n{region_name}:')
    max_val = None
    min_val = None
    for i in np.arange(length):
        data = xr.open_dataset(basedrive+expt[i], decode_times=False)
        # final year
        bsf_last_year = data.barotropicStreamfunction.isel(nTime = np.arange(time-13,time))
        # mask and convert to Sv
        bsf_last_year = bsf_last_year.where(bsf_last_year < netcdf_fill)/1e6
        # mean over the final year
        bsf = bsf_last_year.mean('nTime')
        x = bsf.nx * 2. + 321.
        y = bsf.ny * 2. + 1
        bsf = bsf.where(x >= bounds[0])
        bsf = bsf.where(x <= bounds[1])
        bsf = bsf.where(y >= bounds[2])
        bsf = bsf.where(y <= bounds[3])
        label = f'{labels[i]}:'
        max_bsf = bsf.max().values
        min_bsf = bsf.min().values
        if labels[i] != 'COCO':
            if max_val is None or max_val < min_bsf:
                max_val = min_bsf
            if min_val is None or min_val > min_bsf:
                min_val = min_bsf
        print(f'   {label: <20} {max_bsf:.3g} Sv')
        print(f'   {"": <19} {min_bsf:.3g} Sv')

    print(f'   {0.5 * (min_val + max_val):.2g} $\pm$ {0.5 * (max_val - min_val):.2g} Sv')



Grounding Line:
   COCO:                0.287 Sv
                       -0.295 Sv
   FVCOM:               0.0888 Sv
                       -0.0303 Sv
   MITgcm BAS:          0.0166 Sv
                       -0.0338 Sv
   MITgcm BAS Coupled:  0.0234 Sv
                       -0.028 Sv
   MITgcm JPL:          0.0313 Sv
                       -0.00679 Sv
   MOM6 SIGMA ZSTAR:    0.0055 Sv
                       -0.0187 Sv
   MOM6:                0.0225 Sv
                       -0.0129 Sv
   MPAS-Ocean:          0.00771 Sv
                       -0.0144 Sv
   NEMO-CNRS:           0.018 Sv
                       -0.0125 Sv
   NEMO-UKESM1is:       0.0224 Sv
                       -0.0252 Sv
   POP2x:               0.0231 Sv
                       -0.0122 Sv
   ROMSUTAS:            0.0519 Sv
                       -0.0189 Sv
   0.047 $\pm$ 0.042 Sv

Outer Cavity:
   COCO:                1.16 Sv
                       -0.0451 Sv
   FVCOM:               0.0147 Sv
                       -0.0829